# __Fetch aoe2recs.com match details parallely__

### __Load libraries__

In [2]:
from pymongo import MongoClient

from tqdm import tqdm

from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
import os
import math
import random
import time

from itertools import cycle
import requests
import mechanize
import http.cookiejar
import random
import warnings

import multiprocessing
from joblib import Parallel, delayed

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### __Source functions__

In [ ]:
%run aoe2rec_api_modules.ipynb

### __Fetch match details__

In [ ]:
fetch_multiple_matches_iterate_aoe2recs(db_conn = setup_mongo_conn(), collect = 100000, batch_size = 20)